In [1]:
from qiskit import *
import matplotlib.pyplot as plt
from qiskit import Aer
from qiskit.quantum_info import partial_trace, Statevector
import numpy as np

In [2]:
def encoded_state(sv):
    # traces out ancilla qubits

    # full_statevector = Statevector(cir)
    partial_dm = partial_trace(sv, [5, 6, 7, 8])
    partial_sv = np.diagonal(partial_dm)

    return partial_sv

In [3]:
def stabilizer_measurements(qr, cr):
    gens = [
        ['x','z','z','x','i'],
        ['i','x','z','z','x'],
        ['x','i','x','z','z'],
        ['z','x','i','x','z']
    ]

    cir = QuantumCircuit(qr, cr)
    
    for i, s in enumerate(gens):
        cir.h(i+5)
        for j, g in enumerate(s):
            if (g == 'x'):
                cir.cnot(i+5, j)
            elif (g == 'z'):
                cir.cz(i+5, j)
            else:
                pass
        cir.h(i+5)
        cir.measure(i+5,i)
        # cir.reset(i+5)

    return cir

def encoding(qr):
    cir = QuantumCircuit(qr)

    cir.h(0)
    cir.s(0)
    cir.cz(0,1)
    cir.cz(0,3)
    cir.cy(0,4)

    cir.h(1)
    cir.cz(1,2)
    cir.cz(1,3)
    cir.cx(1,4)

    cir.h(2)
    cir.cz(2,0)
    cir.cz(2,1)
    cir.cx(2,4)

    cir.h(3)
    cir.s(3)
    cir.cz(3,0)
    cir.cz(3,2)
    cir.cy(3,4)
    
    return cir

def error_correction(qr, syn):
    # syn in the error syndrome returned from measuring the stabilizers
    # cir.id(0).c_if(syn, 0)
    cir = QuantumCircuit(qr, syn)

    cir.x(0).c_if(syn, 8) # 8 or 1
    cir.y(0).c_if(syn, 13) # 13 or 11
    cir.z(0).c_if(syn, 5) # 5 or 10

    cir.x(1).c_if(syn, 1) # 1 or 8
    cir.y(1).c_if(syn, 11) # 11 or 13
    cir.z(1).c_if(syn, 10) # 10 or 5

    cir.x(2).c_if(syn, 3) # 3 or 12
    cir.y(2).c_if(syn, 7) # 7 or 14
    cir.z(2).c_if(syn, 4) # 4 or 2

    cir.x(3).c_if(syn, 6)
    cir.y(3).c_if(syn, 15)
    cir.z(3).c_if(syn, 9)

    cir.x(4).c_if(syn, 12) # 12 or 3
    cir.y(4).c_if(syn, 14) # 14 or 7
    cir.z(4).c_if(syn, 2) # 2 or 4

    return cir

In [4]:
q = QuantumRegister(9)
c = ClassicalRegister(4)
cir = QuantumCircuit(q, c)


cir += encoding(q)

# bit flip error on the 0th qubit
cir.ry(np.pi/6, 0)

# cir += stabilizer_measurements(q, c)

# cir += error_correction(q, c)

cir += encoding(q).inverse() # decoding

cir.draw()

C:\Users\nfber\anaconda3\envs\qc\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  
C:\Users\nfber\AppData\Roaming\Python\Python37\site-packages\qiskit\circuit\quantumcircuit.py:876: DeprecationWarning: The QuantumCircuit.extend() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  return self.extend(rhs)


┌───┐┌───┐                                                ┌─────────┐»
q0_0: ┤ H ├┤ S ├─■──■────────■────────────────────■───────────■─┤ RY(π/6) ├»
      └───┘└───┘ │  │ ┌───┐  │                    │           │ └─────────┘»
q0_1: ───────────■──┼─┤ H ├──┼───■──■────────■────┼───■───────┼────────────»
                    │ └───┘  │   │  │ ┌───┐  │    │   │       │            »
q0_2: ──────────────┼────────┼───■──┼─┤ H ├──┼────■───■───■───┼──────■─────»
                    │        │      │ ├───┤  │  ┌───┐     │   │      │     »
q0_3: ──────────────■────────┼──────■─┤ H ├──┼──┤ S ├─────┼───■──────■─────»
                           ┌─┴─┐      └───┘┌─┴─┐└───┘   ┌─┴─┐              »
q0_4: ─────────────────────┤ Y ├───────────┤ X ├────────┤ X ├──────────────»
                           └───┘           └───┘        └───┘              »
q0_5: ─────────────────────────────────────────────────────────────────────»
                                                                           »
q0_6: ─────────────────────────────────────────────────────────────────────»
                                                                           »
q0_7: ─────────────────────────────────────────────────────────────────────»
                                                                           »
q0_8: ─────────────────────────────────────────────────────────────────────»
                                                                           »
c0: 4/═════════════════════════════════════════════════════════════════════»
                                                                           »
«                                                                       ┌─────┐»
«q0_0: ──────────────■───────────────■─────────────────■──────■───────■─┤ SDG ├»
«                    │               │                 │      │ ┌───┐ │ └─────┘»
«q0_1: ──────────────┼─────────■─────┼────■────────■───┼───■──┼─┤ H ├─■────────»
«                    │         │     │    │  ┌───┐ │   │   │  │ └───┘          »
«q0_2: ───────────■──┼───■─────■─────■────┼──┤ H ├─┼───┼───■──┼────────────────»
«                 │  │   │  ┌─────┐┌───┐  │  └───┘ │   │      │                »
«q0_3: ──■────■───■──■───┼──┤ SDG ├┤ H ├──┼────────■───┼──────■────────────────»
«      ┌─┴─┐┌─┴─┐      ┌─┴─┐└─────┘└───┘┌─┴─┐        ┌─┴─┐                     »
«q0_4: ┤ Y ├┤ Y ├──────┤ X ├────────────┤ X ├────────┤ Y ├─────────────────────»
«      └───┘└───┘      └───┘            └───┘        └───┘                     »
«q0_5: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_6: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_7: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«q0_8: ────────────────────────────────────────────────────────────────────────»
«                                                                              »
«c0: 4/════════════════════════════════════════════════════════════════════════»
«                                                                              »
«      ┌───┐
«q0_0: ┤ H ├
«      └───┘
«q0_1: ─────
«           
«q0_2: ─────
«           
«q0_3: ─────
«           
«q0_4: ─────
«           
«q0_5: ─────
«           
«q0_6: ─────
«           
«q0_7: ─────
«           
«q0_8: ─────
«           
«c0: 4/═════
«

In [5]:
backend = Aer.get_backend('statevector_simulator')
job_sim = backend.run(transpile(cir, backend), shots=1000)
result_sim = job_sim.result()
counts = result_sim.get_counts(cir)
sv = result_sim.get_statevector(cir, decimals=3)

In [6]:
# state doesn't return to |00000>, due to the error we forgot about. Instead we have some superposition

encoded_state(sv)

array([0.933156+0.j, 0.      +0.j, 0.      +0.j, 0.      +0.j,
       0.      +0.j, 0.      +0.j, 0.      +0.j, 0.      +0.j,
       0.      +0.j, 0.      +0.j, 0.      +0.j, 0.      +0.j,
       0.      +0.j, 0.      +0.j, 0.      +0.j, 0.      +0.j,
       0.      +0.j, 0.      +0.j, 0.      +0.j, 0.      +0.j,
       0.      +0.j, 0.      +0.j, 0.      +0.j, 0.      +0.j,
       0.      +0.j, 0.      +0.j, 0.      +0.j, 0.      +0.j,
       0.      +0.j, 0.067081+0.j, 0.      +0.j, 0.      +0.j])

In [7]:
q = QuantumRegister(9)
c = ClassicalRegister(4)
cir = QuantumCircuit(q, c)


cir += encoding(q)

# bit flip error on the 0th qubit
cir.rx(np.pi/6, 0)


# now we perform error correction
cir += stabilizer_measurements(q, c)

cir += error_correction(q, c)

cir += encoding(q).inverse() # decoding

cir.draw()

┌───┐┌───┐                                                ┌─────────┐»
q28_0: ┤ H ├┤ S ├─■──■────────■────────────────────■───────────■─┤ RX(π/6) ├»
       └───┘└───┘ │  │ ┌───┐  │                    │           │ └─────────┘»
q28_1: ───────────■──┼─┤ H ├──┼───■──■────────■────┼───■───────┼────────────»
                     │ └───┘  │   │  │ ┌───┐  │    │   │       │            »
q28_2: ──────────────┼────────┼───■──┼─┤ H ├──┼────■───■───■───┼──────■─────»
                     │        │      │ ├───┤  │  ┌───┐     │   │      │     »
q28_3: ──────────────■────────┼──────■─┤ H ├──┼──┤ S ├─────┼───■──────■─────»
                            ┌─┴─┐      └───┘┌─┴─┐└───┘   ┌─┴─┐              »
q28_4: ─────────────────────┤ Y ├───────────┤ X ├────────┤ X ├──────────────»
       ┌───┐                └───┘           └───┘        └───┘              »
q28_5: ┤ H ├────────────────────────────────────────────────────────────────»
       ├───┤                                                                »
q28_6: ┤ H ├────────────────────────────────────────────────────────────────»
       ├───┤                                                                »
q28_7: ┤ H ├────────────────────────────────────────────────────────────────»
       ├───┤                                                                »
q28_8: ┤ H ├────────────────────────────────────────────────────────────────»
       └───┘                                                                »
 c1: 4/═════════════════════════════════════════════════════════════════════»
                                                                            »
«            ┌───┐   ┌───┐                                                     »
«q28_0: ─────┤ X ├───┤ X ├─────────■───────────────────────────────────────────»
«            └─┬─┘   └─┬─┘   ┌───┐ │         ┌───┐                             »
«q28_1: ───────┼───■───┼─────┤ X ├─┼─────────┤ X ├─────────────────────────────»
«              │   │   │     └─┬─┘ │         └─┬─┘        ┌───┐                »
«q28_2: ───────┼───┼───┼───■───┼───┼───────■───┼──────────┤ X ├────────────────»
«              │   │   │   │   │   │ ┌───┐ │   │          └─┬─┘                »
«q28_3: ──■────┼───┼───┼───┼───┼───┼─┤ X ├─┼───┼────────■───┼───────────■──────»
«       ┌─┴─┐  │   │   │   │   │   │ └─┬─┘ │   │        │   │     ┌───┐ │      »
«q28_4: ┤ Y ├──┼───┼───┼───┼───┼───┼───┼───┼───┼────────┼───┼─────┤ X ├─┼──────»
«       └───┘  │   │   │   │   │   │   │   │   │  ┌───┐ │   │  ┌─┐└─┬─┘ │      »
«q28_5: ───────■───■───┼───■───┼───┼───■───┼───┼──┤ H ├─┼───┼──┤M├──┼───┼──────»
«                      │       │   │       │   │  └───┘ │   │  └╥┘  │   │ ┌───┐»
«q28_6: ───────────────┼───────■───┼───────■───┼────────■───┼───╫───■───┼─┤ H ├»
«                      │           │           │            │   ║       │ └───┘»
«q28_7: ───────────────■───────────┼───────────┼────────────■───╫───────■──────»
«                                  │           │                ║              »
«q28_8: ───────────────────────────■───────────■────────────────╫──────────────»
«                                                               ║              »
« c1: 4/════════════════════════════════════════════════════════╩══════════════»
«                                                               0              »
«                                      ┌───┐  ┌───┐   ┌───┐                »
«q28_0: ───────────────────────────────┤ X ├──┤ Y ├───┤ Z ├────────────────»
«                                      └─╥─┘  └─╥─┘   └─╥─┘  ┌───┐  ┌───┐  »
«q28_1: ─────────────────────────────────╫──────╫───────╫────┤ X ├──┤ Y ├──»
«                                        ║      ║       ║    └─╥─┘  └─╥─┘  »
«q28_2: ─────────────────────────────────╫──────╫───────╫──────╫──────╫────»
«          ┌───┐                         ║      ║       ║      ║      ║    »
«q28_3: ───┤ X ├─────────────────────────╫──────╫───────╫──────╫──────╫────»
«          └─┬─┘                         ║      ║   

In [8]:
backend = Aer.get_backend('statevector_simulator')
job_sim = backend.run(transpile(cir, backend), shots=1000)
result_sim = job_sim.result()
counts = result_sim.get_counts(cir)
sv = result_sim.get_statevector(cir, decimals=3)

In [9]:
# back to |00000>, so we corrected the error. QECC can fix arbitrary rotations too

encoded_state(sv)

array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])